# Catalog Search Agent - Find Datasets with Natural Language

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/catalog_search_agent.ipynb)

This notebook demonstrates how to use the Catalog Search Agent to find datasets in data catalogs using natural language queries.

The Catalog Agent can:
- Search through data catalogs (TSV, CSV, JSON formats)
- **Search by geographic region** (location names or bounding boxes)
- Understand natural language queries
- Filter by keywords, dataset type, and provider
- Return structured results with metadata
- Work with Earth Engine Data Catalog and custom catalogs

Uncomment the following line to install [geoai](https://opengeoai.org) if needed.

## Installation

In [ ]:
# %pip install "geoai-py[agents]"

## Import libraries

In [ ]:
import json
from geoai.agents import (
    CatalogAgent,
    CatalogTools,
    create_ollama_model,
    create_openai_model,
    create_anthropic_model,
)

## Load Earth Engine Data Catalog

The Earth Engine Data Catalog is available in **JSON format** from the [opengeos/Earth-Engine-Catalog](https://github.com/opengeos/Earth-Engine-Catalog) repository.

**Important:** Use the JSON format (not TSV) to enable spatial search capabilities with bounding box information.

In [ ]:
catalog_url = "https://raw.githubusercontent.com/opengeos/Earth-Engine-Catalog/refs/heads/master/gee_catalog.json"

## Create a model

You can create a model with the following functions:

- `create_ollama_model`: Create a model using Ollama. You will need to install [Ollama](https://ollama.com/download) separately and pull the model you want to use, such as `llama3.1`.
- `create_openai_model`: Create a model using OpenAI. You will need an OpenAI API key. Set it in the `OPENAI_API_KEY` environment variable.
- `create_anthropic_model`: Create a model using Anthropic. You will need an Anthropic API key. Set it in the `ANTHROPIC_API_KEY` environment variable.

In [ ]:
model = create_ollama_model(model="llama3.1")

## Use CatalogTools Directly (Fast)

For faster searches without LLM overhead, you can use `CatalogTools` directly:

In [ ]:
tools = CatalogTools(catalog_url=catalog_url)

### Get catalog statistics

In [ ]:
stats = json.loads(tools.get_catalog_stats())
print(f"Total datasets: {stats['total_datasets']}")
print(f"\nDataset types:")
for dtype, count in stats.get("dataset_types", {}).items():
    print(f"  {dtype}: {count}")

### Search for datasets by keyword

In [ ]:
result = json.loads(tools.search_datasets(keywords="landcover", max_results=5))
print(f"Found {result['dataset_count']} datasets\n")

for ds in result["datasets"]:
    print(f"ID: {ds['id']}")
    print(f"Title: {ds['title']}")
    print(f"Provider: {ds.get('provider', 'N/A')}")
    print("-" * 80)

### Search by geographic region (NEW!)

Find datasets covering a specific location or bounding box:

In [ ]:
# Search by location name
result = json.loads(
    tools.search_by_region(location="California", keywords="elevation", max_results=5)
)

print(f"Found {result['dataset_count']} datasets covering California\n")

for ds in result["datasets"]:
    print(f"ID: {ds['id']}")
    print(f"Title: {ds['title']}")
    print(f"Bbox: {ds.get('bbox', 'N/A')}")
    print("-" * 80)

In [ ]:
# Search by bounding box coordinates
# San Francisco Bay Area: [west, south, east, north]
result = json.loads(
    tools.search_by_region(
        bbox=[-122.5, 37.5, -122.0, 38.0], keywords="landcover", max_results=3
    )
)

print(f"Found {result['dataset_count']} datasets\n")

for ds in result["datasets"]:
    print(f"ID: {ds['id']}")
    print(f"Title: {ds['title']}")
    print("-" * 80)

### Geocode location names

In [ ]:
# Convert location name to bounding box
location_info = json.loads(tools.geocode_location("New York City"))
print(f"Name: {location_info['name']}")
print(f"Bbox: {location_info['bbox']}")
print(f"Center: {location_info['center']}")

### Search with filters

In [ ]:
result = json.loads(
    tools.search_datasets(
        keywords="elevation", dataset_type="image", provider="NASA", max_results=5
    )
)

print(f"Found {result['dataset_count']} datasets\n")

for ds in result["datasets"]:
    print(f"ID: {ds['id']}")
    print(f"Title: {ds['title']}")
    print(f"Type: {ds.get('type', 'N/A')}")
    print("-" * 80)

### List available providers

In [ ]:
providers = json.loads(tools.list_providers())
print(f"Total providers: {providers['count']}\n")
print("Sample providers:")
for p in providers["providers"][:10]:
    print(f"  - {p}")

### List dataset types

In [ ]:
types = json.loads(tools.list_dataset_types())
print(f"Available dataset types: {types['types']}")

## Use CatalogAgent with Natural Language (LLM)

For natural language queries, create a `CatalogAgent` that uses an LLM to understand and execute searches:

In [ ]:
agent = CatalogAgent(model=model, catalog_url=catalog_url)

### Ask natural language questions

In [ ]:
response = agent.ask("Find datasets about landcover from NASA")
print(response)

In [ ]:
response = agent.ask("Show me elevation data")
print(response)

### Spatial search with natural language (NEW!)

In [ ]:
response = agent.ask("Find landcover datasets covering California")
print(response)

In [ ]:
response = agent.ask("Show me elevation data for San Francisco")
print(response)

In [ ]:
response = agent.ask("Find land cover datasets from NASA that cover New York City")
print(response)

In [ ]:
response = agent.ask("What types of datasets are available?")
print(response)

In [ ]:
response = agent.ask("Find image collections about forests")
print(response)

In [ ]:
response = agent.ask("Find landcover datasets from 2022 onwards")
print(response)

### Get structured results programmatically

In [ ]:
datasets = agent.search_datasets(keywords="sentinel", max_results=5)

for ds in datasets:
    print(f"ID: {ds['id']}")
    print(f"Title: {ds['title']}")
    print(f"Provider: {ds.get('provider', 'N/A')}")
    if ds.get("snippet"):
        print(f"Code: {ds['snippet']}")
    print("-" * 80)